In [17]:
import pandas as pd
import numpy as np
import yfinance as yf
from array import *

dataCompile() compiles all of the equities passed through the paramter and places them into a dataframe. It also removes any NaN values as well as sorts the prices so that it is most recent to least recent

In [73]:
ETFs = ["IVV", "VB", "VO", "VTI", "VWO", "VXUS", "BSV", "BIV", "BLV", "GLD", "SLV", "VDE", "VNQ"]
def dataCompile(etf_list):
    rawPricesdf = pd.DataFrame()

    for ticker in etf_list:
        data = yf.download(ticker)
        flippedData = data['Adj Close'].iloc[::-1]
        flippedData.name = ticker 
        rawPricesdf = pd.concat([rawPricesdf, flippedData], axis=1)
        rawPricesdf.dropna(inplace = True)

    return rawPricesdf
    
compiledData = dataCompile(ETFs)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Historical Simulation Risk Model 